# Classificação dos Reviews

Decidi treinar um classificador porque, uma vez ajustado, ele prediz milhares de reviews a custo quase zero e em milissegundos, evitando as chamadas caras e lentas a um LLM para cada exemplo; além disso, apliquei uma estratégia híbrida: começar com rótulos fracos derivados do score para cobrir rapidamente todo o conjunto e depois refinar o modelo com um subconjunto de rótulos “gold” gerados pelo LLM, aproveitando sua alta precisão apenas onde faz diferença

In [52]:
from dotenv import load_dotenv
from pathlib import Path
from openai import OpenAI

load_dotenv(dotenv_path=Path().resolve().parent / ".env");

In [44]:
from fastmcp import Client

async with Client("https://2b54-2804-14c-71-4f0f-a920-d13d-4dd6-53f1.ngrok-free.app/mcp/") as c:
    result = await c.call_tool("product_sentiment", {"product_id": "a62e25e09e05e6faf31d90c6ec1aa3d1"})


print(result)

[TextContent(type='text', text='{\n  "sentiment": "positivo",\n  "summary": "Os reviews do produto destacam uma mistura de experiências positivas e negativas. Muitos usuários expressam satisfação com a qualidade do produto e a execução do pedido, mencionando que tudo estava dentro do esperado e que recomendariam a compra. No entanto, há várias reclamações relacionadas à entrega, incluindo recebimento parcial de pedidos, como no caso de relógios, onde alguns clientes receberam apenas um dos itens solicitados ou produtos diferentes do que esperavam. Também foi mencionada a falta de instruções claras para a utilização do produto, o que gerou confusão entre os usuários. Além disso, alguns relatam problemas com a condição da embalagem ao receber o produto.",\n  "sentiment_distrib": {\n    "Positivo": 0.68,\n    "Negativo": 0.21,\n    "Neutro": 0.11\n  },\n  "positive_points": [\n    "Produto atende às expectativas",\n    "Satisfação na execução do pedido",\n    "Produto é considerado ótimo"

## Testando com API Gemini

In [77]:
from google import genai
from google.genai import types
from fastmcp import Client
import os

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
mcp_client = Client(
    "http://localhost:8001/mcp/"
)

## Testando com API OpenAI
A API da openAI só irá funcionar se o MCP estiver rodando com o métrodo de transport SSE

In [76]:
prompt = (
        "Qual é o sentimento geral do produto "
        "a62e25e09e05e6faf31d90c6ec1aa3d1?"
    )

async with mcp_client:                             # gerencia a sessão MCP
    response = await client.aio.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config=types.GenerateContentConfig(
            tools=[mcp_client.session],
            temperature=0.15,
            max_output_tokens=512,
        ),
    )

print(response.text)

O sentimento geral do produto é positivo, com 68% de avaliações positivas, 21% negativas e 11% neutras. Os pontos positivos mais mencionados são a satisfação com a qualidade do produto, a execução do pedido e o fato de atender às expectativas. Os pontos negativos incluem problemas com a entrega, como recebimento parcial de pedidos e falta de itens, além de insatisfação com a falta de instruções e o estado da embalagem. As três principais revisões destacam a entrega antes do prazo e a satisfação com o produto e a compra.


In [ ]:
from openai import OpenAI

# Your server URL (replace with your actual URL)
url = "https://2b54-2804-14c-71-4f0f-a920-d13d-4dd6-53f1.ngrok-free.app/mcp"

client = OpenAI()

resp = client.responses.create(
    model="gpt-4.1",
    tools=[
        {
            "type": "mcp",
            "server_label": "sentiment_product",
            "server_url": f"{url}/",
            "require_approval": "never",
        },
    ],
    input="Qual é o sentimento geral do produto a62e25e09e05e6faf31d90c6ec1aa3d1",
)

print(resp.output_text)


APIStatusError: Error code: 424 - {'error': {'message': "Error retrieving tool list from MCP server: 'dice_server'. Http status code: 404 (Not Found)", 'type': 'external_connector_error', 'param': 'tools', 'code': 'http_error'}}